# Tutorial: Build a regression model with Open Datasets

In this tutorial, you leverage the convenience of Azure Open Datasets to create a regression model to predict NYC taxi fare prices. Easily download publicly available taxi, holiday and weather data to create a dataset that can train a regression model using sklearn.

In [28]:
from azureml.opendatasets import NycTlcGreen
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta

pd.options.mode.chained_assignment = None

## Download Data
Begin by downloading the NYC Taxi dataset from Azure Open Datasets. In non-Spark environments, Open Datasets only allows one month of data at a time with certain classes to avoid MemoryError with large datasets. To download 1 year of taxi data, we will fetch 2000 random samples from each month.

Note: Open Datasets has mirroring classes for working in Spark where data size and memory are not a concern.

In [29]:
start = datetime.strptime("1/1/2016","%m/%d/%Y")
end = datetime.strptime("1/31/2016","%m/%d/%Y")

green_taxi_df = pd.concat([NycTlcGreen(start + relativedelta(months=x), end + relativedelta(months=x)) \
        .to_pandas_dataframe().sample(2000) for x in range(12)])
green_taxi_df

[Info] read from C:\Users\FKHOSH~1\AppData\Local\Temp\tmpviwf6gni\https%3A\%2Fazureopendatastorage.azurefd.net\nyctlc\green\puYear=2016\puMonth=1\part-00119-tid-4753095944193949832-fee7e113-666d-4114-9fcb-bcd3046479f3-2689-1.c000.snappy.parquet
[Info] read from C:\Users\FKHOSH~1\AppData\Local\Temp\tmp6e1co7l5\https%3A\%2Fazureopendatastorage.azurefd.net\nyctlc\green\puYear=2016\puMonth=2\part-00060-tid-4753095944193949832-fee7e113-666d-4114-9fcb-bcd3046479f3-2630-2.c000.snappy.parquet
[Info] read from C:\Users\FKHOSH~1\AppData\Local\Temp\tmpd5lgxojh\https%3A\%2Fazureopendatastorage.azurefd.net\nyctlc\green\puYear=2016\puMonth=3\part-00196-tid-4753095944193949832-fee7e113-666d-4114-9fcb-bcd3046479f3-2766-1.c000.snappy.parquet
[Info] read from C:\Users\FKHOSH~1\AppData\Local\Temp\tmpela340gr\https%3A\%2Fazureopendatastorage.azurefd.net\nyctlc\green\puYear=2016\puMonth=4\part-00121-tid-4753095944193949832-fee7e113-666d-4114-9fcb-bcd3046479f3-2691-1.c000.snappy.parquet
[Info] read from C:\

,vendorID,lpepPickupDatetime,lpepDropoffDatetime,passengerCount,tripDistance,puLocationId,doLocationId,pickupLongitude,pickupLatitude,dropoffLongitude,...,paymentType,fareAmount,extra,mtaTax,improvementSurcharge,tipAmount,tollsAmount,ehailFee,totalAmount,tripType
1379860,2,2016-01-14 06:39:00,2016-01-14 06:44:55,1,1.23,None,None,-73.911827,40.775372,-73.899635,...,2,6.5,0.0,0.5,0.3,0.00,0.0,NaN,7.30,1.0
377548,2,2016-01-01 06:22:01,2016-01-01 06:27:14,5,0.91,None,None,-73.962044,40.709797,-73.946716,...,2,5.5,0.0,0.5,0.3,0.00,0.0,NaN,6.30,1.0
473976,2,2016-01-08 20:55:49,2016-01-08 21:05:50,6,3.42,None,None,-73.904823,40.741776,-73.878815,...,2,11.5,0.5,0.5,0.3,0.00,0.0,NaN,12.80,1.0
1246683,2,2016-01-15 08:27:41,2016-01-15 08:41:05,1,3.99,None,None,-73.911484,40.854698,-73.881821,...,2,15.0,0.0,0.5,0.3,0.00,0.0,NaN,15.80,1.0
1152261,2,2016-01-09 04:35:21,2016-01-09 04:41:02,1,0.98,None,None,-73.921776,40.767071,-73.933136,...,1,6.0,0.5,0.5,0.3,0.70,0.0,NaN,8.00,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
998273,1,2016-12-24 22:03:25,2016-12-24 22:17:16,1,5.30,74,235,NaN,NaN,NaN,...,2,16.5,0.5,0.5,0.3,0.00,0.0,NaN,17.80,1.0
857200,2,2016-12-03 20:33:53,2016-12-03 20:53:51,1,4.81,83,258,NaN,NaN,NaN,...,1,18.5,0.5,0.5,0.3,3.00,0.0,NaN,22.80,1.0
607768,2,2016-12-18 16:17:54,2016-12-18 16:33:13,3,2.02,95,56,NaN,NaN,NaN,...,2,11.5,0.0,0.5,0.3,0.00,0.0,NaN,12.30,1.0
78687,2,2016-12-06 09:24:43,2016-12-06 09:41:09,1,9.51,66,11,NaN,NaN,NaN,...,2,27.5,0.0,0.5,0.3,0.00,0.0,NaN,28.30,1.0


Now that the initial data is loaded, define a function to create various time-based features from the pickup datetime field. This will create new fields for the month number, day of month, day of week, and hour of day. From those, we calculate the sin and cosine transformations to capture the cyclical nature of the variable which will allow the model to factor in time-based seasonality. This function also adds a static feature for the country code to join the holiday data. Use the apply() function on the dataframe to interatively apply this function to each row in the dataframe.

In [30]:
def build_time_features(vector):
    pickup_datetime = vector[0]
    month_num = pickup_datetime.month
    day_of_month = pickup_datetime.day
    day_of_week = pickup_datetime.weekday()
    hour_of_day = pickup_datetime.hour
    country_code = "US"
    hr_sin = np.sin(hour_of_day*(2.*np.pi/24))
    hr_cos = np.cos(hour_of_day*(2.*np.pi/24))
    dy_sin = np.sin(day_of_week*(2.*np.pi/7))
    dy_cos = np.cos(day_of_week*(2.*np.pi/7))
    
    return pd.Series((month_num, day_of_month, day_of_week, hour_of_day, country_code, hr_sin, hr_cos, dy_sin, dy_cos))

green_taxi_df[["month_num", "day_of_month","day_of_week", "hour_of_day", "country_code", "hr_sin", "hr_cos", "dy_sin", "dy_cos"]] = green_taxi_df[["lpepPickupDatetime"]].apply(build_time_features, axis=1)
green_taxi_df

,vendorID,lpepPickupDatetime,lpepDropoffDatetime,passengerCount,tripDistance,puLocationId,doLocationId,pickupLongitude,pickupLatitude,dropoffLongitude,...,tripType,month_num,day_of_month,day_of_week,hour_of_day,country_code,hr_sin,hr_cos,dy_sin,dy_cos
1379860,2,2016-01-14 06:39:00,2016-01-14 06:44:55,1,1.23,None,None,-73.911827,40.775372,-73.899635,...,1.0,1,14,3,6,US,1.000000,6.123234e-17,0.433884,-0.900969
377548,2,2016-01-01 06:22:01,2016-01-01 06:27:14,5,0.91,None,None,-73.962044,40.709797,-73.946716,...,1.0,1,1,4,6,US,1.000000,6.123234e-17,-0.433884,-0.900969
473976,2,2016-01-08 20:55:49,2016-01-08 21:05:50,6,3.42,None,None,-73.904823,40.741776,-73.878815,...,1.0,1,8,4,20,US,-0.866025,5.000000e-01,-0.433884,-0.900969
1246683,2,2016-01-15 08:27:41,2016-01-15 08:41:05,1,3.99,None,None,-73.911484,40.854698,-73.881821,...,1.0,1,15,4,8,US,0.866025,-5.000000e-01,-0.433884,-0.900969
1152261,2,2016-01-09 04:35:21,2016-01-09 04:41:02,1,0.98,None,None,-73.921776,40.767071,-73.933136,...,1.0,1,9,5,4,US,0.866025,5.000000e-01,-0.974928,-0.222521
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
998273,1,2016-12-24 22:03:25,2016-12-24 22:17:16,1,5.30,74,235,NaN,NaN,NaN,...,1.0,12,24,5,22,US,-0.500000,8.660254e-01,-0.974928,-0.222521
857200,2,2016-12-03 20:33:53,2016-12-03 20:53:51,1,4.81,83,258,NaN,NaN,NaN,...,1.0,12,3,5,20,US,-0.866025,5.000000e-01,-0.974928,-0.222521
607768,2,2016-12-18 16:17:54,2016-12-18 16:33:13,3,2.02,95,56,NaN,NaN,NaN,...,1.0,12,18,6,16,US,-0.866025,-5.000000e-01,-0.781831,0.623490
78687,2,2016-12-06 09:24:43,2016-12-06 09:41:09,1,9.51,66,11,NaN,NaN,NaN,...,1.0,12,6,1,9,US,0.707107,-7.071068e-01,0.781831,0.623490


Remove some of the columns that you won't need for modeling or additional feature building. Rename the time field for pickup time, and additionally convert the time to midnight using `pandas.Series.dt.normalize`. This is done to all time features so that the datetime column can be later used as a key when joining datasets together at a daily level of granularity.

In [31]:
columns_to_remove = ["lpepDropoffDatetime", "puLocationId", "doLocationId", "extra", "mtaTax",
                     "improvementSurcharge", "tollsAmount", "ehailFee", "tripType", "rateCodeID", 
                     "storeAndFwdFlag", "paymentType", "fareAmount", "tipAmount"]

green_taxi_df.drop(columns_to_remove, axis=1, inplace=True)

green_taxi_df["datetime"] = green_taxi_df["lpepPickupDatetime"].dt.normalize()
green_taxi_df.head(5)

,vendorID,lpepPickupDatetime,passengerCount,tripDistance,pickupLongitude,pickupLatitude,dropoffLongitude,dropoffLatitude,totalAmount,month_num,day_of_month,day_of_week,hour_of_day,country_code,hr_sin,hr_cos,dy_sin,dy_cos,datetime
1379860,2,2016-01-14 06:39:00,1,1.23,-73.911827,40.775372,-73.899635,40.768333,7.3,1,14,3,6,US,1.000000,6.123234e-17,0.433884,-0.900969,2016-01-14
377548,2,2016-01-01 06:22:01,5,0.91,-73.962044,40.709797,-73.946716,40.706902,6.3,1,1,4,6,US,1.000000,6.123234e-17,-0.433884,-0.900969,2016-01-01
473976,2,2016-01-08 20:55:49,6,3.42,-73.904823,40.741776,-73.878815,40.717625,12.8,1,8,4,20,US,-0.866025,5.000000e-01,-0.433884,-0.900969,2016-01-08
1246683,2,2016-01-15 08:27:41,1,3.99,-73.911484,40.854698,-73.881821,40.882130,15.8,1,15,4,8,US,0.866025,-5.000000e-01,-0.433884,-0.900969,2016-01-15
1152261,2,2016-01-09 04:35:21,1,0.98,-73.921776,40.767071,-73.933136,40.774567,8.0,1,9,5,4,US,0.866025,5.000000e-01,-0.974928,-0.222521,2016-01-09


### Enrich with Holiday Data

Now that the taxi data is downloaded and roughly prepared, add in holiday data as additional features. Holiday-specific features will assist model accuracy, as major holidays are times where taxi demand increases dramatically and supply becomes limited. The holiday dataset is relatively small, so fetch the full set by using the `PublicHolidays` class constructor with no parameters for filtering. Preview the data to check the format.

In [32]:
from azureml.opendatasets import PublicHolidays

# call default constructor to download full dataset
holidays_df = PublicHolidays().to_pandas_dataframe()
holidays_df.head(5)

[Info] read from C:\Users\FKHOSH~1\AppData\Local\Temp\tmpya4i60qp\https%3A\%2Fazureopendatastorage.azurefd.net\holidaydatacontainer\Processed\part-00000-tid-8468414522853579044-35925ba8-a227-4b80-9c89-17065e7bf1db-649-c000.snappy.parquet


,countryOrRegion,holidayName,normalizeHolidayName,isPaidTimeOff,countryRegionCode,date
19375,Argentina,Año Nuevo [New Year's Day],Año Nuevo [New Year's Day],None,AR,2008-01-01
19376,Australia,New Year's Day,New Year's Day,None,AU,2008-01-01
19377,Austria,Neujahr,Neujahr,None,AT,2008-01-01
19378,Belarus,Новый год,Новый год,None,BY,2008-01-01
19379,Belgium,Nieuwjaarsdag,Nieuwjaarsdag,None,BE,2008-01-01


Rename the `countryRegionCode` and `date` columns to match the respective field names from the taxi data, and also normalize the time so it can be used as a key. Next, join the holiday data with the taxi data by performing a left-join using the Pandas `merge()` function. This will preserve all records from `green_taxi_df`, but add in holiday data where it exists for the corresponding `datetime` and `country_code`, which in this case is always `\"US\"`. Preview the data to verify that they were merged correctly.

In [33]:
holidays_df = holidays_df.rename(columns={"countryRegionCode": "country_code"})
holidays_df["datetime"] = holidays_df["date"].dt.normalize()

holidays_df.drop(["countryOrRegion", "holidayName", "date"], axis=1, inplace=True)

taxi_holidays_df = pd.merge(green_taxi_df, holidays_df, how="left", on=["datetime", "country_code"])
taxi_holidays_df[taxi_holidays_df["normalizeHolidayName"].notnull()]

,vendorID,lpepPickupDatetime,passengerCount,tripDistance,pickupLongitude,pickupLatitude,dropoffLongitude,dropoffLatitude,totalAmount,month_num,...,day_of_week,hour_of_day,country_code,hr_sin,hr_cos,dy_sin,dy_cos,datetime,normalizeHolidayName,isPaidTimeOff
1,2,2016-01-01 06:22:01,5,0.91,-73.962044,40.709797,-73.946716,40.706902,6.30,1,...,4,6,US,1.000000,6.123234e-17,-0.433884,-0.900969,2016-01-01,New Year's Day,True
25,2,2016-01-01 06:14:43,1,2.44,-73.993576,40.681519,-73.999596,40.655930,10.30,1,...,4,6,US,1.000000,6.123234e-17,-0.433884,-0.900969,2016-01-01,New Year's Day,True
27,2,2016-01-01 16:06:33,1,4.57,-73.962509,40.687862,-73.981361,40.732758,22.25,1,...,4,16,US,-0.866025,-5.000000e-01,-0.433884,-0.900969,2016-01-01,New Year's Day,True
44,2,2016-01-18 11:46:27,1,16.10,-73.925522,40.827877,-73.934982,40.681278,50.30,1,...,0,11,US,0.258819,-9.659258e-01,0.000000,1.000000,2016-01-18,Martin Luther King Jr. Day,None
45,2,2016-01-01 10:41:39,1,3.33,-73.962891,40.711971,-73.918060,40.736832,12.80,1,...,4,10,US,0.500000,-8.660254e-01,-0.433884,-0.900969,2016-01-01,New Year's Day,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23868,2,2016-12-25 00:21:23,1,2.36,NaN,NaN,NaN,NaN,12.30,12,...,6,0,US,0.000000,1.000000e+00,-0.781831,0.623490,2016-12-25,Christmas Day,True
23892,2,2016-12-25 14:05:48,1,1.05,NaN,NaN,NaN,NaN,12.30,12,...,6,14,US,-0.500000,-8.660254e-01,-0.781831,0.623490,2016-12-25,Christmas Day,True
23942,1,2016-12-26 01:43:57,1,0.80,NaN,NaN,NaN,NaN,7.55,12,...,0,1,US,0.258819,9.659258e-01,0.000000,1.000000,2016-12-26,Christmas Day,True
23978,2,2016-12-26 03:38:33,1,1.55,NaN,NaN,NaN,NaN,8.30,12,...,0,3,US,0.707107,7.071068e-01,0.000000,1.000000,2016-12-26,Christmas Day,True


### Enrich with weather data

Now NOAA surface weather data can be appended to the taxi and holiday data. Use a similar approach to fetch the weather data by downloading one month at a time iteratively. Additionally, specify the `cols` parameter with an array of strings to filter the columns to download. This is a very large dataset containing weather surface data from all over the world, so before appending each month, filter the lat/long fields to near NYC using the `query()` function on the dataframe. This will ensure the `weather_df` doesn't get too large.

In [34]:
from azureml.opendatasets import NoaaIsdWeather
start = datetime.strptime("1/1/2016","%m/%d/%Y")
end = datetime.strptime("1/31/2016","%m/%d/%Y")

weather_df = pd.concat([NoaaIsdWeather(cols=["temperature", "precipTime", "precipDepth"], start_date=start + relativedelta(months=x), end_date=end + relativedelta(months=x))\
        .to_pandas_dataframe().query("latitude>=40.53 and latitude<=40.88 and longitude>=-74.09 and longitude<=-73.72 and temperature==temperature") for x in range(12)])

[Info] read from C:\Users\FKHOSH~1\AppData\Local\Temp\tmpi7x_kizu\https%3A\%2Fazureopendatastorage.azurefd.net\isdweatherdatacontainer\ISDWeather\year=2016\month=1\part-00000-tid-738723883827836859-85e0759b-51fa-4430-84ec-cb8ab6a57033-2623-1.c000.snappy.parquet
[Info] read from C:\Users\FKHOSH~1\AppData\Local\Temp\tmpi7x_kizu\https%3A\%2Fazureopendatastorage.azurefd.net\isdweatherdatacontainer\ISDWeather\year=2016\month=1\part-00001-tid-738723883827836859-85e0759b-51fa-4430-84ec-cb8ab6a57033-2628-1.c000.snappy.parquet
[Info] read from C:\Users\FKHOSH~1\AppData\Local\Temp\tmpi7x_kizu\https%3A\%2Fazureopendatastorage.azurefd.net\isdweatherdatacontainer\ISDWeather\year=2016\month=1\part-00002-tid-738723883827836859-85e0759b-51fa-4430-84ec-cb8ab6a57033-2622-1.c000.snappy.parquet
[Info] read from C:\Users\FKHOSH~1\AppData\Local\Temp\tmpi7x_kizu\https%3A\%2Fazureopendatastorage.azurefd.net\isdweatherdatacontainer\ISDWeather\year=2016\month=1\part-00003-tid-738723883827836859-85e0759b-51fa-44

In [36]:
weather_df

,wban,latitude,temperature,usaf,datetime,longitude,precipDepth,precipTime
204647,14732,40.783,2.8,725030,2016-01-02 03:00:00,-73.867,NaN,NaN
204670,14732,40.779,-4.4,725030,2016-01-22 13:51:00,-73.880,0.0,1.0
204694,14732,40.779,5.0,725030,2016-01-08 02:51:00,-73.880,0.0,1.0
204701,14732,40.779,-1.1,725030,2016-01-04 15:51:00,-73.880,0.0,1.0
204715,14732,40.779,4.4,725030,2016-01-01 21:51:00,-73.880,0.0,1.0
...,...,...,...,...,...,...,...,...
1248471,94728,40.789,4.4,725053,2016-12-23 13:51:00,-73.967,0.0,1.0
1248555,94728,40.789,5.0,725053,2016-12-12 13:51:00,-73.967,0.0,1.0
1248580,94728,40.789,3.9,725053,2016-12-18 07:01:00,-73.967,NaN,NaN
1248597,94728,40.789,7.8,725053,2016-12-25 00:51:00,-73.967,0.0,1.0


Again call `pandas.Series.dt.normalize` on the `datetime` field in the weather data so it matches the time key in `taxi_holidays_df`.


Next group the weather data to have daily aggregated weather values. Define a dict `aggregations` to define how to aggregate each field at a daily level. For`temperature` take the mean and for `precipTime` and `precipDepth` take the daily maximum. Use the `groupby()` function along with the aggregations to group the data. Preview the data to ensure there is one record per day.

In [37]:
weather_df["datetime"] = weather_df["datetime"].dt.normalize()

# group by datetime
aggregations = {"precipTime": "max", "temperature": "mean", "precipDepth": "max"}
weather_df_grouped = weather_df.groupby("datetime").agg(aggregations)
weather_df_grouped.head(10)

,precipTime,temperature,precipDepth
datetime,,,
2016-01-01,1.0,5.197345,0.0
2016-01-02,1.0,2.567857,0.0
2016-01-03,1.0,3.846429,0.0
2016-01-04,1.0,0.123894,0.0
2016-01-05,6.0,-7.206250,0.0
2016-01-06,6.0,-0.896396,0.0
2016-01-07,6.0,3.180645,0.0
2016-01-08,1.0,4.384091,0.0
2016-01-09,6.0,6.710274,3.0


Note: The examples in this tutorial merge data using Pandas functions and custom aggregations, but the Open Datasets SDK has classes designed to easily merge and enrich data sets. See the [notebook](https://github.com/Azure/OpenDatasetsNotebooks/blob/master/tutorials/data-join/04-nyc-taxi-join-weather-in-pandas.ipynb) for code examples of these design patterns.

### Cleanse data

Merge the existing taxi and holiday data with the new weather data. This time `datetime` is the only key, and again perform a left-join of the data. Run the `describe()` function on the new dataframe to see summary statistics for each field.

In [38]:
taxi_holidays_weather_df = pd.merge(taxi_holidays_df, weather_df_grouped, how="left", on=["datetime"])
taxi_holidays_weather_df.describe()

,vendorID,passengerCount,tripDistance,pickupLongitude,pickupLatitude,dropoffLongitude,dropoffLatitude,totalAmount,month_num,day_of_month,day_of_week,hour_of_day,hr_sin,hr_cos,dy_sin,dy_cos,precipTime,temperature,precipDepth
count,24000.000000,24000.000000,24000.000000,12000.000000,12000.000000,12000.000000,12000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000,2.400000e+04,24000.000000,24000.000000,24000.000000,24000.000000,24000.000000
mean,1.789667,1.355292,2.830398,-73.814393,40.678791,-73.837019,40.690729,14.668251,6.500000,15.068750,3.247792,13.582875,-0.239687,-1.510585e-02,-0.079292,-0.059630,13.318667,13.878272,1037.956292
std,0.407554,1.020018,3.118302,3.016385,1.663152,2.698609,1.488032,11.738532,3.452124,8.477555,1.951209,6.708372,0.667528,7.048175e-01,0.714457,0.692640,10.333162,9.484443,2788.844868
min,1.000000,0.000000,0.000000,-74.164825,0.000000,-75.186440,0.000000,-200.000000,1.000000,1.000000,0.000000,0.000000,-1.000000,-1.000000e+00,-0.974928,-0.900969,1.000000,-13.379464,0.000000
25%,2.000000,1.000000,1.040000,-73.961370,40.693539,-73.967514,40.695128,7.880000,3.750000,8.000000,2.000000,9.000000,-0.866025,-7.071068e-01,-0.781831,-0.900969,6.000000,6.620773,0.000000
50%,2.000000,1.000000,1.840000,-73.947132,40.745928,-73.945869,40.745914,11.300000,6.500000,15.000000,3.000000,15.000000,-0.500000,-1.836970e-16,0.000000,-0.222521,6.000000,13.108323,10.000000
75%,2.000000,1.000000,3.500000,-73.919638,40.802049,-73.913059,40.791076,17.750000,9.250000,22.000000,5.000000,19.000000,0.258819,7.071068e-01,0.781831,0.623490,24.000000,22.944737,127.000000
max,2.000000,7.000000,106.680000,0.000000,41.081047,0.000000,41.081055,450.000000,12.000000,30.000000,6.000000,23.000000,1.000000,1.000000e+00,0.974928,1.000000,24.000000,31.303665,9999.000000


From the summary statistics, you see that there are several fields that have outliers or values that will reduce model accuracy. First filter the lat/long fields to be within the same bounds you used for filtering weather data. The `tripDistance` field has some bad data, because the minimum value is negative. The `passengerCount` field has bad data as well, with the max value being 210 passengers. Lastly, the `totalAmount` field has negative values, which don't make sense in the context of our model.

Filter out these anomolies using query functions, and then remove the last few columns unnecesary for training.

Note: since a random sample of 2000 was taken for each month of the taxi data, the statistics may vary each time this is ran.

In [39]:
final_df = taxi_holidays_weather_df.query("pickupLatitude>=40.53 and pickupLatitude<=40.88 and \
                                           pickupLongitude>=-74.09 and pickupLongitude<=-73.72 and \
                                           tripDistance>0 and tripDistance<75 and \
                                           passengerCount>0 and passengerCount<100 and \
                                           totalAmount>0")

Call `describe()` again on the data to ensure cleansing worked as expected. The final data is prepared and cleansed, consisting of taxi, holiday, and weather data, and is ready to use for machine learning model training.

In [40]:
final_df.describe()

,vendorID,passengerCount,tripDistance,pickupLongitude,pickupLatitude,dropoffLongitude,dropoffLatitude,totalAmount,month_num,day_of_month,day_of_week,hour_of_day,hr_sin,hr_cos,dy_sin,dy_cos,precipTime,temperature,precipDepth
count,11763.000000,11763.000000,11763.000000,11763.000000,11763.000000,11763.000000,11763.000000,11763.000000,11763.000000,11763.000000,11763.000000,11763.000000,11763.000000,1.176300e+04,11763.000000,11763.000000,11763.000000,11763.000000,11763.000000
mean,1.790190,1.369294,2.841407,-73.937911,40.746224,-73.910901,40.730818,14.557917,3.501318,14.929270,3.252317,13.538553,-0.236544,-2.265927e-03,-0.070226,-0.059059,11.993964,10.288261,192.179546
std,0.407191,1.041634,2.829864,0.041121,0.056818,1.364114,0.753468,9.989165,1.707350,8.475793,1.948127,6.778012,0.668812,7.048492e-01,0.718871,0.689122,10.114775,8.530011,1223.101074
min,1.000000,1.000000,0.010000,-74.035194,40.572906,-74.183029,0.000000,0.010000,1.000000,1.000000,0.000000,0.000000,-1.000000,-1.000000e+00,-0.974928,-0.900969,1.000000,-13.379464,0.000000
25%,2.000000,1.000000,1.090000,-73.961601,40.693594,-73.967793,40.695440,8.160000,2.000000,8.000000,2.000000,9.000000,-0.866025,-7.071068e-01,-0.781831,-0.900969,1.000000,3.504580,0.000000
50%,2.000000,1.000000,1.900000,-73.947517,40.745842,-73.946243,40.745789,11.300000,4.000000,15.000000,3.000000,15.000000,-0.500000,-1.836970e-16,0.000000,-0.222521,6.000000,10.468276,3.000000
75%,2.000000,1.000000,3.530000,-73.920509,40.801752,-73.913807,40.789942,17.380000,5.000000,22.000000,5.000000,19.000000,0.258819,7.071068e-01,0.781831,0.623490,24.000000,16.966923,41.000000
max,2.000000,6.000000,38.850000,-73.738899,40.879982,0.000000,41.073185,123.800000,6.000000,30.000000,6.000000,23.000000,1.000000,1.000000e+00,0.974928,1.000000,24.000000,26.524107,9999.000000


## Train a model

The data is ready to train a machine learning model.

In [41]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import r2_score, mean_absolute_percentage_error, mean_squared_error

### Training Function

Define a function that can be used to create a model pipeline that can be trained and then used for scoring. This pipeline has 2 steps: preprocessing and model training.

<b>Preprocessing Stages:</b>
The preprocessing step of the pipeline also has 2 stages, one for numerical features and one for categorical features.
For the numerical features, let's fill in any blanks with 0's. While the training data may not have any nulls in the these fields, future data that is scored may and this step will take care of those for us. Optionally, a scaler transformation could be added in this step as well. Similarly for the categorical variables, let's have the null values filled with "MISSING". Additionally to the categorical variables, these will need to be one hot encoded, so we will include that step in our pipeline.

<b>Model Training Stage:</b>
An input parameter will determine which type of model of train. Let's test out a linear regression and random forest model to start. 

The two steps are put together into the pipeline which is what the function is returning.

In [49]:
def createClassModel(algo_name, catg, nums):
  numeric_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='constant', fill_value=0))])
  
  categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='constant', fill_value="MISSING")), ('onehot', OneHotEncoder(handle_unknown='ignore'))])
  
  preprocesser = ColumnTransformer(transformers=[('num', numeric_transformer, nums), ('cat', categorical_transformer, catg)])
  
  if algo_name == 'linear_regression':
    model=Ridge(alpha=100)
  elif algo_name == 'random_forest':
    model = RandomForestRegressor()
  else:
    pass
  ModelPipeline = Pipeline(steps=[('preprocessor', preprocesser), ("model", model)])
  return ModelPipeline

Let's define the arguments that will be passed to the function. `catg_cols` is a list of the categorical variables that will be transformed in our processing step. `num_cols` is a list of the numerical variables that will be transformed in our processing step. Let's define the target column as `label` so it can be used in future steps as well.

In [43]:
catg_cols = ["vendorID", "month_num", "day_of_month", "normalizeHolidayName", "isPaidTimeOff"]
num_cols = ["passengerCount", "tripDistance", "precipTime", "temperature", "precipDepth", "hr_sin", "hr_cos", "dy_sin", "dy_cos"]
label = ["totalAmount"]

The training is ready to begin, but first, let's make sure that the categorical variables are strings in our dataframe to ensure no errors in our pipeline. 

Next, the data is split into training and test sets by using the `train_test_split()` function in the `scikit-learn` library. The `test_size` parameter determines the percentage of data to allocate to testing. The `random_state` parameter sets a seed to the random number generator, so that your train-test splits are deterministic.

The training will happen in the for loop so that both algorithms can be tested. The createClassModel funtion is called to retreive the pipeline that can then be trained using the training dataset. 

Once trained, the test dataset is then ran through the model to test the model's performance. Using various functions from sklearn.metrics, the R2 score, MAPE, and RMSE can be used to measure model performance.

In [50]:
# make sure categorical columns are strings
final_df[catg_cols] = final_df[catg_cols].astype("str")

# split data
X_train, X_test, y_train, y_test = train_test_split(final_df.drop(label, axis=1), final_df[label], test_size=0.2, random_state=222)

# test 2 algorithms
for algorithmname in ["linear_regression", 'random_forest']:
    fitPipeline = createClassModel(algorithmname, catg_cols, num_cols) # get pipeline
    fitPipeline.fit(X_train, y_train.values.ravel())                   # fit pipeine

    y_pred = fitPipeline.predict(X_test)                               # score with fitted pipeline

    # Evaluate
    r2 = r2_score(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    print(algorithmname)
    print("R2:", r2)
    print("MAPE:", mape)
    print("RMSE:", rmse)
    print()

linear_regression
R2: 0.8034971051723139
MAPE: 0.15888983234876766
RMSE: 4.606544019524053

random_forest
R2: 0.8073017231520601
MAPE: 0.14715914748857337
RMSE: 4.5617309259357475

